In [261]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np
import random
from datetime import date
import time

""" Заполнили сведения из Headers при обращении к https://auto.ru/cars/used/ 
    (здесь указаны мои, полагаю, лучше использовать ваши)
    Не забудьте внести cookies"""
cookie = ''
headers = {
    'authority': 'auto.ru',
    'method': 'GET',
    'path': '/cars/used/',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'cookie': cookie,
    'priority': 'u=0, i',
    'referer': 'https://auto.ru/cars/new/',
    'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform':'"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
}


In [262]:
def get_car(link):   
    
    """Функция собирает сведения со странички объявления"""

    url_car = link # записали ссылку

    # загрузили страницу с данными по автомобилю
    page_auto = requests.get(link, headers=headers)
        
    # создали дерево
    soup_auto = BeautifulSoup(page_auto.content, 'html.parser')

    # извлекли данные из блока CardBreadcrumbs
    CardBreadcrums = soup_auto.find('div', {'class': 'CardBreadcrumbs'})
    car_make = CardBreadcrums.find_all('a')[2].text
    car_model = CardBreadcrums.find_all('a')[3].text
    car_gen = CardBreadcrums.find_all('a')[4].text
    car_type = CardBreadcrums.find_all('a')[5].text
    car_compl = CardBreadcrums.find_all('a')[6].text

    # извлекли данные об объявлении
    ann_date = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__creationDate'}).text
    ann_id = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__id'}).text
    car_price = soup_auto.find('span', {'class': 'OfferPriceCaption__price'}).text.replace('\xa0','')
    ann_city = soup_auto.find('span', {'class': 'MetroListPlace__regionName MetroListPlace_nbsp'}).text
        
    # Нашли нужный блок с информацией, ищем только по нему, бережем ресурсы
    card_info = soup_auto.find('div', {'class': 'CardInfo-ateuv'})

    # записали ссылку на комплектацию
    link_cpl = card_info.find('a').get('href')
        
    # извлекли сведения об автомобиле
    avail = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_availability'}).find_all('div')[1].text
    year = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_year'}).find_all('div')[1].text
    mileage = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_kmAge'}).find_all('div')[1].text
    color = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_color'}).find_all('div')[1].text
    if card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')[2] == 'Электро':
        eng_power, eng_power_kw, eng_type = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        eng_power_kw = int(eng_power_kw.replace('\xa0',' ').split()[0])
        eng_size = None
        pow_resrv = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_electricRange'}).find_all('div')[1].text.split()[0]
    else:
        eng_size, eng_power, eng_type = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        eng_size = float(eng_size.split()[0])
        eng_power_kw = None
        pow_resrv = None
    options = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_complectationOrEquipmentCount'}).find_all('div')[1].text
    transmission = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_transmission'}).find_all('div')[1].text
    drive = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_drive'}).find_all('div')[1].text
    st_wheel= card_info.find('li', {'class': 'CardInfoRow CardInfoRow_wheel'}).find_all('div')[1].text
    condition = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_state'}).find_all('div')[1].text
    count_owner = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_ownersCount'}).find_all('div')[1].text
    try:
        original_pts = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_pts'}).find_all('div')[1].text
    except:
        original_pts = None
    try:
        customs = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_customs'}).find_all('div')[1].text
    except:
        customs = None

    # подготовили словарь для месяцев
    dict_month = {
        'января': 1,
        'февраля': 2,
        'марта': 3,
        'апреля': 4,
        'мая': 5,
        'июня': 6,
        'июля': 7,
        'августа': 8,
        'сентября': 9,
        'октября': 10,
        'ноября': 11,
        'декабря':12
    }

    # записали данные в словарь
    row = {
        'url_car': url_car,
        'car_make': car_make,
        'car_model': car_model,
        'car_gen': car_gen,
        'car_type': car_type,
        'car_compl': car_compl,
        'ann_date': str(date(2024, dict_month[ann_date.split()[1]], int(ann_date.split()[0]))),
        'ann_id': ann_id[2:],
        'car_price': int(car_price[:-1]),
        'ann_city': ann_city,
        'link_cpl': link_cpl,
        'avail': avail.replace('\xa0',' '),
        'year': int(year),
        'mileage': int(mileage.replace('\xa0','')[:-3]),
        'color': color,
        'eng_size': eng_size,
        'eng_power': int(eng_power.replace('\xa0',' ').split()[0]),
        'eng_power_kw': eng_power_kw,
        'eng_type': eng_type,
        'pow_resrv': pow_resrv,
        'options': options.replace('\xa0',' '),
        'transmission': transmission,
        'drive': drive,
        'st_wheel': st_wheel,
        'condition': condition,
        'count_owner': count_owner.replace('\xa0',' '),
        'original_pts': original_pts,
        'customs': customs
        }
    
    # передаем словарь
    return row


In [263]:
def get_page(p):

    """ Функция собирает ссылки на авто со странички 
        и запускает функцию сбора информации по каждому найденному объявлению"""

    # изготовили ссылку на страницу списка с объявлениями
    url = f'https://auto.ru/cars/used/?page={p}'

    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # построили дерево
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # нашли в нём ссылки на авто, записали в список
    urls = []
    for link in soup.find_all('a'):
        if link.get('href') != None and 'cars/used/sale' in link.get('href')  and link.get('href') not in urls:
            urls.append(link.get('href'))

    # прошли по ссылкам записали данные в список словарей
    info = []
    mistakes = []
    random_seconds = [3, 5, 7, 10, 12]
    for link in tqdm(urls):
        time.sleep(random.choice(random_seconds)) # немного подождали
        try:
            info.append(get_car(link))
        except:
            print(f'Ошибка при загрузке страницы: {link}')
            mistakes.append(link)

    return info

In [264]:
for page in tqdm(range(2, 3)):
    try:
        date_page = get_page(page)
        # дописали данные в файл
        with open('data.txt', 'a') as data:
            for record in date_page:
                print(record, file=data)
        # записали сведения в лог
        with open('log.txt', 'a') as log:
            print(f'Успешно загружена страница {page}. Число добавленных записей {len(date_page)}', file=log)
    except:
        # записали сведения в лог
        with open('log.txt', 'a') as log:
            print(f'Не была загружена страница {page}. Число добавленных записей {0}', file=log)
    
    # закрыли файлы
    data.close()
    log.close()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

In [265]:
# Смотрим на пример того, что приходит
date_page[0]

{'url_car': 'https://auto.ru/cars/used/sale/kia/sportage/1125369058-091c668e/',
 'car_make': 'Kia',
 'car_model': 'Sportage',
 'car_gen': 'III',
 'car_type': 'Внедорожник 5 дв.',
 'car_compl': '2.0 MT (150 л.с.) 4WD',
 'ann_date': '2024-10-08',
 'ann_id': '1125369058',
 'car_price': 1080000,
 'ann_city': 'Москва',
 'link_cpl': 'https://auto.ru/catalog/cars/kia/sportage/6407700/6407720/specifications/6407720_6410941_6410862/',
 'avail': 'В наличии',
 'year': 2012,
 'mileage': 320889,
 'color': 'белый',
 'eng_size': 2.0,
 'eng_power': 150,
 'eng_power_kw': None,
 'eng_type': 'Бензин',
 'pow_resrv': None,
 'options': '9 опций',
 'transmission': 'механическая',
 'drive': 'полный',
 'st_wheel': 'Левый',
 'condition': 'Не требует ремонта',
 'count_owner': '2 владельца',
 'original_pts': 'Оригинал',
 'customs': 'Растаможен'}